
# 目标

通过实战

1. 理解EFK工作过程
2. 基于Serilog构建一个简单的Asp.Net Core 程序，并在Kibana中可视化分析

本文主体参照多篇文章并局部调整而成，网上文章大部分行不通

## 前置条件

1. Kubernete 已安装，并且已启动，此处使用Minikube
2. Kubectl 已安装
3. Helm 已安装


## 工具说明
    强烈建议使用VS Code，并安装Remote for VS Code(如果操作远程服务器的话), Docker for VS Code, K8s for VS Code

    另外，VS Code 的Terminal非常好用

## 前置软件安装

本文基于Debian

### 安装kubectl
参考： https://kubernetes.io/docs/tasks/tools/install-kubectl-linux/#install-using-native-package-management

#### 程序安装：

```bash
sudo apt-get update
sudo apt-get install -y apt-transport-https ca-certificates curl
sudo curl -fsSLo /usr/share/keyrings/kubernetes-archive-keyring.gpg https://packages.cloud.google.com/apt/doc/apt-key.gpg
echo "deb [signed-by=/usr/share/keyrings/kubernetes-archive-keyring.gpg] https://apt.kubernetes.io/ kubernetes-xenial main" | sudo tee /etc/apt/sources.list.d/kubernetes.list
sudo apt-get update
sudo apt-get install -y kubectl
```

#### bash 自动完成配置：

```bash

echo 'source <(kubectl completion bash)' >>~/.bashrc

```

#### bash 短命令配置：

```bash

echo 'alias k=kubectl' >>~/.bashrc
echo 'complete -F __start_kubectl k' >>~/.bashrc

```

### 安装Minikube

参考：https://minikube.sigs.k8s.io/docs/start/

#### 安装

```bash

curl -LO https://storage.googleapis.com/minikube/releases/latest/minikube_latest_amd64.deb
sudo dpkg -i minikube_latest_amd64.deb

```

#### 启动

```bash

minikube start

```

#### 基本使用
```bash

kubectl get po -A
minikube kubectl -- get po -A

minikube dashboard

```

### 安装Helm

参考：https://helm.sh/docs/intro/install/

```bash

curl https://baltocdn.com/helm/signing.asc | sudo apt-key add -
sudo apt-get install apt-transport-https --yes
echo "deb https://baltocdn.com/helm/stable/debian/ all main" | sudo tee /etc/apt/sources.list.d/helm-stable-debian.list
sudo apt-get update
sudo apt-get install helm

```

## 配置过程




### 配置 Elasticsearch

#### 调整Minikube，启用必要插件

参考 https://github.com/elastic/helm-charts/tree/master/elasticsearch/examples/minikube

TODO 不确定是否必须

```bash
#minikube addons enable default-storageclass
#minikube addons enable storage-provisioner
#minikube addons enable ingress
```

#### 安装Elasticsearch
生成修订文件
```yaml
#elasticsearch_values.yaml
---
# Permit co-located instances for solitary minikube virtual machines.
antiAffinity: "soft"
```

```bash
helm repo add elastic https://helm.elastic.co
helm repo update
helm install elasticsearch elastic/elasticsearch -f elasticsearch_values.yaml
```

#### 自测Elasticsearch
终端1
```bash
kubectl port-forward svc/elasticsearch-master 9200
```
终端2
```bash
curl localhost:9200/_cat/indices
```

### 配置Kibana
#### 安装Kibana

```bash
helm install kibana elastic/kibana 
```

#### 自测Kibana

终端1
```bash
kubectl port-forward deployment/kibana-kibana 5601 
```

终端1
```bash
open https://localhost:5601
```

### 配置Counter Pod

Add counter.yaml
```yaml
## counter.yaml
apiVersion: v1
kind: Pod
metadata:
  name: counter
spec:
  containers:
  - name: count
    image: busybox
    args: [/bin/sh, -c, 'i=0; while true; do echo "Demo log $i: $(date)"; i=$((i+1)); sleep 1; done']
```

启动 Pod

```bash
kubectl apply -f counter.yaml
```

### 配置Fluentd(WIP)

基于官方调整 https://github.com/fluent/fluentd-kubernetes-daemonset/blob/master/fluentd-daemonset-elasticsearch-rbac.yaml


```bash
wget https://github.com/fluent/fluentd-kubernetes-daemonset/blob/master/fluentd-daemonset-elasticsearch-rbac.yaml

# 关闭TLS
#           - name: FLUENT_ELASTICSEARCH_SSL_VERIFY
#            value: "true"
# 调整ELASTICSEARCH_HOST
#          - name:  FLUENT_ELASTICSEARCH_HOST
#            value: "elasticsearch-master"
```

### 验证示例C#程序

#### 创建示例代码

```bash
mkdir dotnet-console-sample
cd dotnet-console-sample/

dotnet new console

dotnet add package Serilog.Sinks.Console 
dotnet add package Serilog.Enrichers.Context 
dotnet add package Serilog.Enrichers.EnrichedProperties 
dotnet add package Serilog.Enrichers.Thread

cat > Program.cs <<'EOF'
using System.Threading.Tasks;
using Serilog;

Log.Logger = new LoggerConfiguration()
 .Enrich.WithThreadId()
 .WriteTo.Console()
    .CreateLogger();

Log.Information("--begin--");

int counter=1;
do{
    Log.Information("counter: {counter}", counter);
    await Task.Delay(TimeSpan.FromSeconds(1));
    counter++;

}while(true);


Log.Information("--end--");

Log.CloseAndFlush();
EOF

cat > Dockerfile <<'EOF'
FROM mcr.microsoft.com/dotnet/runtime:6.0 AS base
WORKDIR /app

FROM mcr.microsoft.com/dotnet/sdk:6.0 AS build
WORKDIR /src
COPY ["dotnet-console-sample.csproj", "./"]
RUN dotnet restore "dotnet-console-sample.csproj"
COPY . .
WORKDIR "/src/"
RUN dotnet build "dotnet-console-sample.csproj" -c Release -o /app/build

FROM build AS publish
RUN dotnet publish "dotnet-console-sample.csproj" -c Release -o /app/publish

FROM base AS final
WORKDIR /app
COPY --from=publish /app/publish .
ENTRYPOINT ["dotnet", "dotnet-console-sample.dll"]

EOF

cat > .dockerignore <<'EOF'
**/.dockerignore
**/.env
**/.git
**/.gitignore
**/.project
**/.settings
**/.toolstarget
**/.vs
**/.vscode
**/.idea
**/*.*proj.user
**/*.dbmdl
**/*.jfm
**/azds.yaml
**/bin
**/charts
**/docker-compose*
**/Dockerfile*
**/node_modules
**/npm-debug.log
**/obj
**/secrets.dev.yaml
**/values.dev.yaml
LICENSE
README.md
EOF

```

#### 生成镜像

```bash
docker build -t dotnet-console-sample .
```

#### 测试镜像
```bash
docker run --rm -it dotnet-console-sample
```

#### 配置dotnet-console-sample Pod

Add dotnet-console-sample.yaml

```yaml
## dotnet-console-sample.yaml
apiVersion: v1
kind: Pod
metadata:
  name: dotnet-console-sample
spec:
  containers:
  - name: dotnet-console-sample
    image: dotnet-console-sample
    #args: [/bin/sh, -c, 'i=0; while true; do echo "Demo log $i: $(date)"; i=$((i+1)); sleep 1; done']
```

#### 启动Pod

```bash
kubectl apply -f dotnet-console-sample.yaml
```

### 部署metricbeat, 跟踪系统指标(如CPU,内存,网络,磁盘,网络流量等)(可玩项)

```bash
helm install metricbeat elastic/metricbeat

# open you kibana now
# 1. add index pattern like Metricbeat-*
# 2. discovery something

```



## Asp.Net 程序示例

### 构想

### 原型

### 效果

# 参考链接

1. https://techcommunity.microsoft.com/t5/core-infrastructure-and-security/getting-started-with-logging-using-efk-on-kubernetes/ba-p/1333050
2. https://kubernetes.io/docs/tasks/tools/install-kubectl-linux/#install-using-native-package-management
3. https://minikube.sigs.k8s.io/docs/start/
4. https://kubernetes.io/docs/tasks/tools/
5. https://helm.sh/docs/intro/install/
6. https://kubernetes.io/docs/concepts/cluster-administration/logging/
7. https://docs.fluentd.org/container-deployment/kubernetes
8. https://www.elastic.co/beats/metricbeat
9. https://kubernetes.io/docs/tutorials/stateless-application/expose-external-ip-address/
10. https://mherman.org/blog/logging-in-kubernetes-with-elasticsearch-Kibana-fluentd/
11. https://logz.io/blog/deploying-the-elk-stack-on-kubernetes-with-helm/
12. https://medium.com/kubernetes-tutorials/learn-how-to-assign-pods-to-nodes-in-kubernetes-using-nodeselector-and-affinity-features-e62c437f3cf8
13. https://minikube.sigs.k8s.io/docs/handbook/accessing/
14. https://kubernetes.io/docs/concepts/services-networking/service/
15. https://stackoverflow.com/questions/41509439/whats-the-difference-between-clusterip-nodeport-and-loadbalancer-service-types
17. https://github.com/elastic/helm-charts/tree/master/metricbeat
18. https://gist.githubusercontent.com/vineet68sharma/bdbd8a96f162ef119e9bc66bd47e6b8d/raw/f84a28535c204cd82aaf1eaf1d2265cd64d0d1e0/efk9.yaml
19. https://github.com/fluent/fluentd-kubernetes-daemonset/blob/master/fluentd-daemonset-elasticsearch-rbac.yaml




